In [ ]:
# This complete file (Extension Implementation) was coded by 
# Mohith Akhilesh and Pragyanta Dhal
# mdhulipalla@umass.edu pdhal@umass.edu 



In [1]:
#Import required libraries

import pandas as pd
import numpy as np

In [2]:
#Load the data, add headers, write back the data

df = pd.read_csv("./Downloads/adult_data.csv")
headers = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship",
           "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "salary_range"]
df.to_csv("./Downloads/adult_data.csv", header = headers, index = False)
df = pd.read_csv("./Downloads/adult_data.csv")

print(len(df))

32560


In [3]:
#Perform preprocessing. Alter the marital_status column and replace all null values with np.nan

married = [' Married-civ-spouse',' Married-spouse-absent', ' Married-AF-spouse', ' Separated', ' Widowed']
unmarried = [' Divorced',' Never-married' ]

df['marital_status'] = df['marital_status'].replace(married, 'married')
df['marital_status'] = df['marital_status'].replace(unmarried, 'unmarried')
df = df.replace(' ?', np.nan).dropna(axis=0, how='any', subset = headers)
df.to_csv('~/adult_data_preprocessed.csv', index = False)

print(len(df))

30161


In [4]:
#Split data into partitions. This will be used in Pruning optimization. The number of partitions is assumed to be 10

import csv
import os
def split_csv(source_filepath, dest_folder, split_file_prefix,
                records_per_file):
    """
    Split a source csv into multiple csvs of equal numbers of records,
    except the last file.

    Includes the initial header row in each split file.

    Split files follow a zero-index sequential naming convention like so:

        `{split_file_prefix}_0.csv`
    """
    if records_per_file <= 0:
        raise Exception('records_per_file must be > 0')

    with open(source_filepath, 'r') as source:
        reader = csv.reader(source)
        headers = next(reader)

        file_idx = 0
        records_exist = True

        while records_exist:

            i = 0
            target_filename = f'{split_file_prefix}_{file_idx}.csv'
            target_filepath = os.path.join(dest_folder, target_filename)

            with open(target_filepath, 'w') as target:
                writer = csv.writer(target)

                while i < records_per_file:
                    if i == 0:
                        writer.writerow(headers)

                    try:
                        writer.writerow(next(reader))
                        i += 1
                    except StopIteration:
                        records_exist = False
                        break

            if i == 0:
                # we only wrote the header, so delete that file
                os.remove(target_filepath)

            file_idx += 1





In [5]:
partitions = 10
split_csv("./adult_data_preprocessed.csv", "./splitFiles", "adult_data", len(df)/partitions )